In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_columns',None)
from sklearn.preprocessing import StandardScaler
import joblib

In [3]:
df = pd.read_csv('Predict_Data_Singapore Credit Dataset.csv')

In [4]:
df.head(5)

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_duration,percent_of_income,years_at_residence,age,other_credit,housing,existing_loans_count,job,dependents,phone
0,1 - 200 SGD,48,good,furniture/appliances,5951,< 100 SGD,1 - 4 years,2,2,22,none,own,1,skilled,1,no
1,unknown,36,good,education,9055,unknown,1 - 4 years,2,4,35,none,other,1,unskilled,2,yes
2,1 - 200 SGD,30,critical,car,5234,< 100 SGD,unemployed,4,2,28,none,own,2,management,1,no
3,< 0 SGD,24,critical,car,1199,< 100 SGD,> 7 years,4,4,60,none,own,2,unskilled,1,no
4,< 0 SGD,30,perfect,business,8072,unknown,< 1 year,2,3,25,bank,own,3,skilled,1,no


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   checking_balance      250 non-null    object
 1   months_loan_duration  250 non-null    int64 
 2   credit_history        250 non-null    object
 3   purpose               250 non-null    object
 4   amount                250 non-null    int64 
 5   savings_balance       250 non-null    object
 6   employment_duration   250 non-null    object
 7   percent_of_income     250 non-null    int64 
 8   years_at_residence    250 non-null    int64 
 9   age                   250 non-null    int64 
 10  other_credit          250 non-null    object
 11  housing               250 non-null    object
 12  existing_loans_count  250 non-null    int64 
 13  job                   250 non-null    object
 14  dependents            250 non-null    int64 
 15  phone                 250 non-null    ob

## Importing Libraries

In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns',None)

In [23]:
from sklearn.preprocessing import StandardScaler
import joblib

## Data preparation

#### Merging car0 and car with same value car 

In [8]:
df['purpose'] = np.where(df['purpose']=='car0','car',df['purpose'])

#### Replacing Binary Class with 0 and 1

In [9]:
df['phone'] = np.where(df['phone']=='no',0,df['phone'])
df['phone'] = np.where(df['phone']=='yes',1,df['phone'])

In [11]:
df.sample(3)

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_duration,percent_of_income,years_at_residence,age,other_credit,housing,existing_loans_count,job,dependents,phone
161,unknown,36,poor,business,7980,unknown,< 1 year,4,4,27,none,rent,2,skilled,1,1
163,1 - 200 SGD,36,poor,car,8086,100 - 500 SGD,> 7 years,2,4,42,none,own,4,management,1,1
41,unknown,6,good,furniture/appliances,2978,500 - 1000 SGD,1 - 4 years,1,2,32,none,own,1,skilled,1,1


In [12]:
df.describe()

,months_loan_duration,amount,percent_of_income,years_at_residence,age,existing_loans_count,dependents
count,250.000000,250.000000,250.000000,250.000000,250.00000,250.000000,250.000000
mean,20.860000,3249.960000,3.040000,2.916000,35.24800,1.472000,1.140000
std,12.834144,2882.693788,1.111459,1.077771,11.10639,0.615411,0.347683
min,4.000000,250.000000,1.000000,1.000000,19.00000,1.000000,1.000000
25%,12.000000,1316.500000,2.000000,2.000000,27.00000,1.000000,1.000000
50%,18.000000,2119.500000,4.000000,3.000000,33.00000,1.000000,1.000000
75%,24.000000,3932.500000,4.000000,4.000000,41.00000,2.000000,1.000000
max,72.000000,15653.000000,4.000000,4.000000,68.00000,4.000000,2.000000


### Checking if there is any extra value in our categorical feature

In [13]:
def unique_values(df,columns):
    return np.unique(df[columns])
count = 0
for i in df.columns:
    if df[i].dtypes == 'O':
        print('Feature:-',i,':')
        print(unique_values(df,i))
        count +=1
print('Number of Categorical features:',count)

Feature:- checking_balance :
['1 - 200 SGD' '< 0 SGD' '> 200 SGD' 'unknown']
Feature:- credit_history :
['critical' 'good' 'perfect' 'poor' 'very good']
Feature:- purpose :
['business' 'car' 'education' 'furniture/appliances' 'renovations']
Feature:- savings_balance :
['100 - 500 SGD' '500 - 1000 SGD' '< 100 SGD' '> 1000 SGD' 'unknown']
Feature:- employment_duration :
['1 - 4 years' '4 - 7 years' '< 1 year' '> 7 years' 'unemployed']
Feature:- other_credit :
['bank' 'none' 'store']
Feature:- housing :
['other' 'own' 'rent']
Feature:- job :
['management' 'skilled' 'unemployed' 'unskilled']
Feature:- phone :
[0 1]
Number of Categorical features: 9


### Creating Dummy Variable


In [15]:
catvars = [i for i in df.columns if df[i].dtype=='O'] 
df1 = df
for vars in catvars:
    dummy = pd.get_dummies(df[vars],prefix=vars,drop_first=True)
    df1 = pd.concat([df1,dummy],axis =1)

C:\Users\DARIN\AppData\Local\Temp\ipykernel_15540\3761676731.py:4: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  dummy = pd.get_dummies(df[vars],prefix=vars,drop_first=True)


In [16]:
df.shape,df1.shape,len(catvars)

((250, 16), (250, 43), 9)

In [17]:
df1 = df1.drop(catvars,axis=1)
df1.shape

(250, 34)

In [18]:
df1 = df1.reset_index(drop=True)

### Scaling Features

In [20]:
sc = StandardScaler()
sc_x = sc.fit_transform(df1)
sc_x = pd.DataFrame(sc_x)

## Final Dataset

In [25]:
sc_x.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
0,2.118914,0.938864,-0.937584,-0.851607,-1.195220,-0.768505,-0.403473,-0.629800,-0.234206,-0.789542,0.992032,-0.204124,-0.340693,-0.214535,-0.711354,-0.234206,1.057660,-0.169725,-0.261488,0.789542,-0.261488,-0.487467,-0.410152,-0.389966,-0.598912,-0.224544,0.481173,-0.224544,0.673415,-0.5,0.717741,-0.110208,-0.531085,-0.809709
1,1.182032,2.017795,-0.937584,1.007797,-0.022374,-0.768505,2.478479,-0.629800,-0.234206,1.266557,0.992032,-0.204124,-0.340693,-0.214535,-0.711354,4.269750,-0.945484,-0.169725,-0.261488,-1.266557,-0.261488,2.051422,-0.410152,-0.389966,-0.598912,-0.224544,0.481173,-0.224544,-1.484967,-0.5,-1.393261,-0.110208,1.882938,1.235011
2,0.713591,0.689640,0.865462,-0.851607,-0.653906,0.859684,-0.403473,-0.629800,-0.234206,-0.789542,-1.008032,-0.204124,-0.340693,-0.214535,1.405770,-0.234206,-0.945484,-0.169725,-0.261488,0.789542,-0.261488,-0.487467,-0.410152,-0.389966,-0.598912,4.453463,0.481173,-0.224544,0.673415,-0.5,-1.393261,-0.110208,-0.531085,-0.809709
3,0.245151,-0.712901,0.865462,1.007797,2.233098,0.859684,-0.403473,1.587806,-0.234206,-0.789542,-1.008032,-0.204124,-0.340693,-0.214535,1.405770,-0.234206,-0.945484,-0.169725,-0.261488,0.789542,-0.261488,-0.487467,-0.410152,-0.389966,1.669694,-0.224544,0.481173,-0.224544,0.673415,-0.5,-1.393261,-0.110208,1.882938,-0.809709
4,0.713591,1.676110,-0.937584,0.078095,-0.924563,2.487874,-0.403473,1.587806,-0.234206,-0.789542,-1.008032,4.898979,-0.340693,-0.214535,-0.711354,-0.234206,-0.945484,-0.169725,-0.261488,-1.266557,-0.261488,2.051422,-0.410152,2.564324,-0.598912,-0.224544,-2.078256,-0.224544,0.673415,-0.5,0.717741,-0.110208,-0.531085,-0.809709


## Importing model

In [26]:
model = joblib.load('final_model.pkl')

## Prediction

In [28]:
prediction = model.predict(sc_x)

In [30]:
prediction

array([1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0])

In [32]:
pd.DataFrame(prediction).value_counts()

1    195
0     55
dtype: int64

## Saving the predicted result in csv file

In [37]:
predicted = pd.concat([df,pd.DataFrame(prediction)],axis=1)

In [39]:
predicted.to_csv('predicted_result.csv')